In [78]:
import re
import csv
from pathlib import Path

from datasets import load_dataset

In [3]:
dataset = load_dataset("code_search_net", streaming=True)

In [48]:
def remove_comments_and_docstrings(sample):
    """
    Removes all comments and docstrings from the provided Python code.

    Args:
        code (str): The Python code to be cleaned.

    Returns:
        str: The cleaned Python code without comments or docstrings.
    """

    # Remove single-line comments
    sample["func_code_string"] = re.sub(r"//.*?\n", "", sample["func_code_string"])

    # Remove multi-line comments
    sample["func_code_string"] = re.sub(r"/\*.*?\*/", "", sample["func_code_string"], flags=re.DOTALL)

    # Remove docstrings
    sample["func_code_string"] = re.sub(r'""".*?"""', "", sample["func_code_string"], flags=re.DOTALL)
    sample["func_code_string"] = re.sub(r"''.*?''", "", sample["func_code_string"], flags=re.DOTALL)

    return sample

In [49]:
dataset = dataset.filter(
    lambda example : example['language'] == "python"
    ).map(
        remove_comments_and_docstrings
    )

In [74]:
data_iter = iter(dataset['test'])

In [75]:
sample = next(data_iter)

In [76]:
print(sample["func_code_string"])

def get_vid_from_url(url):
        
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')


In [84]:
outpath = Path("/home/thv200000/projects/automated_code_comment/data")
with open (outpath / "python_comments_train.csv", 'w') as test_file:
    fieldnames = ['code', 'comment']
    writer = csv.DictWriter(test_file, fieldnames=fieldnames)
    writer.writeheader()
    for i, sample in enumerate(dataset["train"]):
        if i == 1000:
            break
        writer.writerow({
            "code": sample["func_code_string"],
            "comment":sample["func_documentation_string"]
        })